In [1]:
from autoencoder import Autoencoder

In [2]:
Autoencoder().build_graph().summary()

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
autoencoder_input (InputLaye [(None, 32)]              0         
_________________________________________________________________
encoder_0 (Dense)            (None, 28)                924       
_________________________________________________________________
encoder_1 (Dense)            (None, 25)                725       
_________________________________________________________________
decoder_0 (DenseTranspose)   (None, 28)                753       
_________________________________________________________________
decoder_1 (DenseTranspose)   (None, 32)                956       
Total params: 1,709
Trainable params: 1,709
Non-trainable params: 0
_________________________________________________________________
